In [1]:
!pip install spike2py
!pip install h5py

1. Loading the data

In [3]:
import h5py

f=h5py.File("C:\\Users\\sinha\\Dropbox\\CPLab\\cpl data\\20230623_dk1_BW_context_day1.mat")
channels = list(f.keys())  # Example: Getting all dataset names if that's what you intended
print(channels)

['V20230623_dk1_BW_context_day1_Ch1', 'V20230623_dk1_BW_context_day1_Ch2', 'V20230623_dk1_BW_context_day1_Ch29', 'V20230623_dk1_BW_context_day1_Ch3', 'V20230623_dk1_BW_context_day1_Ch31', 'V20230623_dk1_BW_context_day1_Ch4', 'V20230623_dk1_BW_context_day1_Ch5', 'V20230623_dk1_BW_context_day1_Ch6', 'V20230623_dk1_BW_context_day1_Ch8', 'file']


In [4]:
channel1 = f[channels[3]]
print(channel1.keys())

<KeysViewHDF5 ['comment', 'interval', 'length', 'offset', 'scale', 'start', 'times', 'title', 'units', 'values']>


In [8]:
for channel_key in channel1.keys():
    print(channel_key, channel1[channel_key][0])

channel1_times=list(channel1['times'])
channel1_data = list(channel1['values'])

print(len(channel1_times[0]), len(channel1_data[0]))

comment 0
interval [0.0005]
length [2695982.]
offset [0.]
scale [0.00015259]
start [5.e-05]
times [5.00000000e-05 5.50000000e-04 1.05000000e-03 ... 1.34798955e+03
 1.34799005e+03 1.34799055e+03]
title [76]
units [86]
values [-0.04989624 -0.06500244 -0.08148193 ... -0.07293701 -0.05187988
 -0.02639771]
2695982 2695982


### 2. Now, according to Matt, I first need to use the butterworth lowpass filter

In [7]:
from scipy.signal import butter, filtfilt, firls, iirnotch, cheby2
from scipy import signal

def butter_lowpass(cutoff, fs, order=4):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a


def butter_lowpass_filter(data, cutoff, fs, order=4, axis=-1):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data, padlen=0, axis=axis)
    return y

In [13]:
print(channel1_data[0])

[-0.04989624 -0.06500244 -0.08148193 ... -0.07293701 -0.05187988
 -0.02639771]


In [15]:
filtered_data= butter_lowpass_filter(channel1_data[0], 200, 20000)
sig_length = len(filtered_data)
rfs=1000
num_samples = int(sig_length * rfs)

filtered_data_resampled = signal.resample(filtered_data, num_samples)

MemoryError: bad allocation

2. Resampling with Notch Filter (from cpl_pipeline)

In [38]:
import mne
import numpy as np

channel1_data = np.array(channel1_data)
channe1_data_filter_resample=mne.filter.resample(channel1_data, 1000)
channe1_data_filter=mne.filter.notch_filter(channe1_data_filter_resample, 1000, [60, 161, 60], filter_length='auto', phase='zero', fir_window='hamming', fir_design='firwin', verbose=None)
def preprocess_raw(raw_signal, fmin=1, fmax=100):
    raw: mne.io.RawArray = raw_signal.copy()
    raw.load_data()
    raw.filter(fmin, fmax, fir_design="firwin")
    raw.resample(1000)
    raw.notch_filter(np.arange(60, 161, 60), fir_design="firwin")
    #raw.set_eeg_reference(ref_channels=["Ref"])
    raw.drop_channels(["Ref"])
    return raw


MemoryError: bad allocation

In [34]:
ch1_processed=preprocess_raw(channel1_data[0])

AttributeError: 'numpy.ndarray' object has no attribute 'load_data'